In [1]:
import os
!export CUDA_VISIBLE_DEVICES=0
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# from urllib.request import urlopen
# from PIL import Image
# import timm
# import torch

# img = Image.open(urlopen(
#     'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
# ))

# model = timm.create_model('mixer_l16_224.goog_in21k_ft_in1k', pretrained=True)
# model = model.eval()

# # get model specific transforms (normalization, resize)
# data_config = timm.data.resolve_model_data_config(model)
# transforms = timm.data.create_transform(**data_config, is_training=False)

# output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

# top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)


In [3]:
# from urllib.request import urlopen
# from PIL import Image
# import timm
# import torch
# img = Image.open(urlopen(
#     'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
# ))

# model = timm.create_model('mixer_b16_224.miil_in21k_ft_in1k', pretrained=True)
# model = model.eval()

# # get model specific transforms (normalization, resize)
# data_config = timm.data.resolve_model_data_config(model)
# transforms = timm.data.create_transform(**data_config, is_training=False)

# output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

# top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)



In [3]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

Extension horovod.torch has not been built: /home/shariff/.local/lib/python3.8/site-packages/horovod/torch/mpi_lib_v2.cpython-39-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.


In [6]:
model = model.cuda()

In [6]:
model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [2]:
from utils import *
dataloader = get_dataloader("imagenet", batch_size=256)


In [7]:
evaluate_model(model, dataloader, count_nonzero_only=True)


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.66%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0044 seconds
model has 59.88 M parameters


In [ ]:
test_accuracy, best_sparse_model_checkpoint = finetune(model, dataloader, num_finetune_epochs=5)

In [4]:
model.load_state_dict(torch.load("best_mlp_base.pt"))

<All keys matched successfully>

In [8]:
@torch.inference_mode()
def evaluate_topk(
    model: nn.Module,
    dataloader: DataLoader, 
    verbose=True,
    device=None,
    topk = (1,)
) -> float:
  # model.eval()
    mega_list = []
    for inputs, targets in tqdm(dataloader, desc="eval", leave=False, 
                                disable=not verbose):
        # Move the data from CPU to GPU
        if device is None:
            inputs = inputs.cuda()
            targets = targets.cuda()
        else:
            inputs = inputs.to(device)
            targets = targets.to(device)

        # Inference
        outputs = model(inputs)
        maxk = max(topk)
        batch_size = targets.size(0)

        _, y_pred = outputs.topk(k=maxk, dim=1) 
        y_pred = y_pred.t()  

        target_reshaped = targets.view(1, -1).expand_as(y_pred)  
        correct = (y_pred == target_reshaped)  

        list_topk_accs = []  # idx is topk1, topk2, ... etc
        for k in topk:
            ind_which_topk_matched_truth = correct[:k]  # [maxk, B] -> [k, B]
            flattened_indicator_which_topk_matched_truth = ind_which_topk_matched_truth.reshape(-1).float()  # [k, B] -> [kB]
            tot_correct_topk = flattened_indicator_which_topk_matched_truth.float().sum(dim=0, keepdim=True)  # [kB] -> [1]
            topk_acc = tot_correct_topk / batch_size  # topk accuracy for entire batch
            list_topk_accs.append(topk_acc)
        mega_list.append(list_topk_accs)
    return mega_list  # list of topk accuracies for entire batch [topk1, topk2, ... etc]


In [9]:
mega_list = evaluate_topk(model, dataloader['val'], topk=(5,))

In [10]:
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.9166360288858414


In [15]:
# model.load_state_dict(best_sparse_model_checkpoint['state_dict'])

<All keys matched successfully>

In [16]:
mega_list = evaluate_topk(model, dataloader['val'], topk=(5,))
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.906640625


In [13]:
evaluate_model(model, dataloader, count_nonzero_only=True)


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.66%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0030 seconds
model has 59.88 M parameters


In [18]:
# torch.save(model.state_dict(), "best_mlp_base.pt")

In [14]:
test_model = copy.deepcopy(model)
test_model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [ ]:
# def collapse_mlp_gelu_layer(layer_id):
    

In [15]:
test_model.blocks[11].mlp_channels.act = nn.LeakyReLU(0.0)

In [16]:
test_model = test_model.cuda()
test_model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [18]:
evaluate_model(test_model, dataloader, count_nonzero_only=True)


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.32%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0032 seconds
model has 59.88 M parameters


In [17]:
test_accuracy, best_sparse_model_checkpoint = finetune(test_model, dataloader, num_finetune_epochs=3)

Finetuning changed Model


    Epoch 1 test Accuracy 74.48% / Best test Accuracy: 74.48%


    Epoch 2 test Accuracy 74.82% / Best test Accuracy: 74.82%


    Epoch 3 test Accuracy 74.40% / Best test Accuracy: 74.82%


    Epoch 4 test Accuracy 74.48% / Best test Accuracy: 74.82%


    Epoch 5 test Accuracy 74.52% / Best test Accuracy: 74.82%


    Epoch 6 test Accuracy 74.36% / Best test Accuracy: 74.82%


    Epoch 7 test Accuracy 74.32% / Best test Accuracy: 74.82%


In [19]:
test_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])

<All keys matched successfully>

In [20]:
for relu in [0.5, 1]:
    compressed_model_1 = copy.deepcopy(test_model)
    compressed_model_1.blocks[11].mlp_channels.act = nn.LeakyReLU(relu)
    evaluate_model(compressed_model_1, dataloader, count_nonzero_only=False)
    test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model_1, dataloader, num_finetune_epochs=3)
    compressed_model_1.load_state_dict(best_sparse_model_checkpoint['state_dict'])
    test_model = copy.deepcopy(compressed_model_1)
    print("At relu slope:", relu, ":", test_accuracy)

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=0.14%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0032 seconds
model has 59.88 M parameters
Finetuning changed Model


    Epoch 1 test Accuracy 74.44% / Best test Accuracy: 74.44%


    Epoch 2 test Accuracy 74.68% / Best test Accuracy: 74.68%


    Epoch 3 test Accuracy 74.32% / Best test Accuracy: 74.68%
At relu slope: 0.5 : [74.43999481201172, 74.68000030517578, 74.31999969482422]


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=73.92%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0031 seconds
model has 59.88 M parameters
Finetuning changed Model


    Epoch 1 test Accuracy 74.50% / Best test Accuracy: 74.50%


    Epoch 2 test Accuracy 73.86% / Best test Accuracy: 74.50%


    Epoch 3 test Accuracy 74.28% / Best test Accuracy: 74.50%
At relu slope: 1 : [74.5, 73.85999298095703, 74.27999877929688]


In [21]:
compressed_model_1

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [24]:
evaluate_model(compressed_model_1, dataloader, count_nonzero_only=True)


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.50%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0031 seconds
model has 59.88 M parameters


In [25]:
w0 = compressed_model_1.blocks[11].mlp_channels.fc1.weight.data
w1 = compressed_model_1.blocks[11].mlp_channels.fc2.weight.data
b0 = compressed_model_1.blocks[11].mlp_channels.fc1.bias.data
b1 = compressed_model_1.blocks[11].mlp_channels.fc2.bias.data
compressed_model_1.blocks[11].mlp_channels.fc2.weight.data = torch.matmul(w1, w0)
compressed_model_1.blocks[11].mlp_channels.fc2.bias.data = torch.matmul(w1, b0) + b1

compressed_model_1.blocks[11].mlp_channels.fc1 = nn.Identity()
compressed_model_1.blocks[11].mlp_channels.act = nn.Identity()
compressed_model_1.blocks[11].mlp_channels.drop1 = nn.Identity()


In [26]:
evaluate_model(compressed_model_1, dataloader, count_nonzero_only=True)


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.50%
model has size=212.66 MiB
model has macs=11.80 Gmacs
average inference time is 0.0031 seconds
model has 55.75 M parameters


In [27]:
compressed_11_gelu_1 = copy.deepcopy(compressed_model_1)

In [11]:
def collapse_mlp_gelu_layer_channel(test_model, layer_id):
    test_model.blocks[layer_id].mlp_channels.act = nn.LeakyReLU(0.0)
    test_model = test_model.cuda()
    print("test_model with leaky relu", test_model)
    evaluate_model(test_model, dataloader, count_nonzero_only=True)
    test_accuracy, best_sparse_model_checkpoint = finetune(test_model, dataloader, num_finetune_epochs=3)
    test_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])
    for relu in [0.5, 1]:
        compressed_model = copy.deepcopy(test_model)
        compressed_model.blocks[layer_id].mlp_channels.act = nn.LeakyReLU(relu)
        evaluate_model(compressed_model, dataloader, count_nonzero_only=False)
        test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model, dataloader, num_finetune_epochs=3)
        compressed_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])
        test_model = copy.deepcopy(compressed_model)
        print("At relu slope:", relu, ":", test_accuracy)
    print("to be compressed_model", compressed_model)
    evaluate_model(compressed_model, dataloader, count_nonzero_only=True)
    w0 = compressed_model.blocks[layer_id].mlp_channels.fc1.weight.data
    w1 = compressed_model.blocks[layer_id].mlp_channels.fc2.weight.data
    b0 = compressed_model.blocks[layer_id].mlp_channels.fc1.bias.data
    b1 = compressed_model.blocks[layer_id].mlp_channels.fc2.bias.data
    compressed_model.blocks[layer_id].mlp_channels.fc2.weight.data = torch.matmul(w1, w0)
    compressed_model.blocks[layer_id].mlp_channels.fc2.bias.data = torch.matmul(w1, b0) + b1
    compressed_model.blocks[layer_id].mlp_channels.fc1 = nn.Identity()
    compressed_model.blocks[layer_id].mlp_channels.act = nn.Identity()
    compressed_model.blocks[layer_id].mlp_channels.drop1 = nn.Identity()
    evaluate_model(compressed_model, dataloader, count_nonzero_only=True)
    return compressed_model



    


In [12]:
def collapse_mlp_gelu_layer_token(test_model, layer_id):
    test_model.blocks[layer_id].mlp_tokens.act = nn.LeakyReLU(0.0)
    test_model = test_model.cuda()
    print("test_model with leaky relu", test_model)
    evaluate_model(test_model, dataloader, count_nonzero_only=True)
    test_accuracy, best_sparse_model_checkpoint = finetune(test_model, dataloader, num_finetune_epochs=3)
    test_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])
    for relu in [0.5, 1]:
        compressed_model = copy.deepcopy(test_model)
        compressed_model.blocks[layer_id].mlp_tokens.act = nn.LeakyReLU(relu)
        evaluate_model(compressed_model, dataloader, count_nonzero_only=False)
        test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model, dataloader, num_finetune_epochs=3)
        compressed_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])
        test_model = copy.deepcopy(compressed_model)
        print("At relu slope:", relu, ":", test_accuracy)
    print("to be compressed_model", compressed_model)
    evaluate_model(compressed_model, dataloader, count_nonzero_only=True)
    w0 = compressed_model.blocks[layer_id].mlp_tokens.fc1.weight.data
    w1 = compressed_model.blocks[layer_id].mlp_tokens.fc2.weight.data
    b0 = compressed_model.blocks[layer_id].mlp_tokens.fc1.bias.data
    b1 = compressed_model.blocks[layer_id].mlp_tokens.fc2.bias.data
    compressed_model.blocks[layer_id].mlp_tokens.fc2.weight.data = torch.matmul(w1, w0)
    compressed_model.blocks[layer_id].mlp_tokens.fc2.bias.data = torch.matmul(w1, b0) + b1
    compressed_model.blocks[layer_id].mlp_tokens.fc1 = nn.Identity()
    compressed_model.blocks[layer_id].mlp_tokens.act = nn.Identity()
    compressed_model.blocks[layer_id].mlp_tokens.drop1 = nn.Identity()
    evaluate_model(compressed_model, dataloader, count_nonzero_only=True)
    return compressed_model



    


In [10]:
# model.blocks[11].mlp_channels.fc1 = nn.Identity()
# model.blocks[11].mlp_channels.act = nn.Identity()
# model.blocks[11].mlp_channels.drop1 = nn.Identity()


In [12]:
# model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [13]:
# compressed_model_1 = copy.deepcopy(compressed_11_gelu_1)
# torch.save(compressed_11_gelu_1.state_dict(), "MLP models/compressed_11_gelu.pt")
# model.load_state_dict(torch.load("MLP models/compressed_11_gelu.pt"))

In [14]:
# test_model.blocks[layer_id].mlp_tokens.act = nn.LeakyReLU(0.0)
# test_model = test_model.cuda()


In [15]:
compressed_11_gelu_1 = collapse_mlp_gelu_layer_channel(model, 11)

test_model with leaky relu MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.26%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0031 seconds
model has 59.88 M parameters
Finetuning changed Model


    Epoch 1 test Accuracy 74.80% / Best test Accuracy: 74.80%


    Epoch 2 test Accuracy 74.48% / Best test Accuracy: 74.80%


...

    Epoch 3 test Accuracy 74.48% / Best test Accuracy: 74.54%
At relu slope: 1 : [73.79999542236328, 74.53999328613281, 74.47999572753906]
to be compressed_model MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, in

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.54%
model has size=228.43 MiB
model has macs=12.61 Gmacs
average inference time is 0.0031 seconds
model has 59.88 M parameters


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.54%
model has size=212.66 MiB
model has macs=11.80 Gmacs
average inference time is 0.0031 seconds
model has 55.75 M parameters


In [16]:
evaluate_model(compressed_11_gelu_1, dataloader, count_nonzero_only=True)

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=74.54%
model has size=212.66 MiB
model has macs=11.80 Gmacs
average inference time is 0.0031 seconds
model has 55.75 M parameters


In [17]:
# torch.save(compressed_11_gelu_1.state_dict(), "MLP models/compressed_11_gelu.pt")


In [1]:
import os
!export CUDA_VISIBLE_DEVICES=0
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from urllib.request import urlopen
from PIL import Image
import timm
import torch

# img = Image.open(urlopen(
#     'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
# ))

# model = timm.create_model('mixer_b16_224.goog_in21k_ft_in1k', pretrained=True)
# model = model.eval()

# # get model specific transforms (normalization, resize)
# data_config = timm.data.resolve_model_data_config(model)
# transforms = timm.data.create_transform(**data_config, is_training=False)

# output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

# top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)
# model = model.cuda()
# model.load_state_dict(torch.load("best_mlp_base.pt"))
from utils import *
dataloader = get_dataloader("imagenet", batch_size=256)
# evaluate_model(model, dataloader, count_nonzero_only=True)


Extension horovod.torch has not been built: /home/shariff/.local/lib/python3.8/site-packages/horovod/torch/mpi_lib_v2.cpython-39-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.


In [2]:
@torch.inference_mode()
def evaluate_topk(
    model: nn.Module,
    dataloader: DataLoader, 
    verbose=True,
    device=None,
    topk = (1,)
) -> float:
  # model.eval()
    mega_list = []
    for inputs, targets in tqdm(dataloader, desc="eval", leave=False, 
                                disable=not verbose):
        # Move the data from CPU to GPU
        if device is None:
            inputs = inputs.cuda()
            targets = targets.cuda()
        else:
            inputs = inputs.to(device)
            targets = targets.to(device)

        # Inference
        outputs = model(inputs)
        maxk = max(topk)
        batch_size = targets.size(0)

        _, y_pred = outputs.topk(k=maxk, dim=1) 
        y_pred = y_pred.t()  

        target_reshaped = targets.view(1, -1).expand_as(y_pred)  
        correct = (y_pred == target_reshaped)  

        list_topk_accs = []  # idx is topk1, topk2, ... etc
        for k in topk:
            ind_which_topk_matched_truth = correct[:k]  # [maxk, B] -> [k, B]
            flattened_indicator_which_topk_matched_truth = ind_which_topk_matched_truth.reshape(-1).float()  # [k, B] -> [kB]
            tot_correct_topk = flattened_indicator_which_topk_matched_truth.float().sum(dim=0, keepdim=True)  # [kB] -> [1]
            topk_acc = tot_correct_topk / batch_size  # topk accuracy for entire batch
            list_topk_accs.append(topk_acc)
        mega_list.append(list_topk_accs)
    return mega_list  # list of topk accuracies for entire batch [topk1, topk2, ... etc]


In [3]:
# mega_list = evaluate_topk(model, dataloader['val'], topk=(5,))

In [4]:
# mega_list = [i[0].item() for i in mega_list]
# print(sum(mega_list)/len(mega_list))

In [5]:
def collapse_mlp_gelu_layer_channel(test_model, layer_id, num_finetune_epochs=3):
    test_model.blocks[layer_id].mlp_channels.act = nn.LeakyReLU(0.0)
    test_model = test_model.cuda()
    print("test_model with leaky relu", test_model)
    evaluate_model(test_model, dataloader, count_nonzero_only=True)
    test_accuracy, best_sparse_model_checkpoint = finetune(test_model, dataloader, num_finetune_epochs=num_finetune_epochs)
    test_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])
    for relu in [0.5, 1]:
        compressed_model = copy.deepcopy(test_model)
        compressed_model.blocks[layer_id].mlp_channels.act = nn.LeakyReLU(relu)
        evaluate_model(compressed_model, dataloader, count_nonzero_only=False)
        test_accuracy, best_sparse_model_checkpoint = finetune(compressed_model, dataloader, num_finetune_epochs=num_finetune_epochs)
        compressed_model.load_state_dict(best_sparse_model_checkpoint['state_dict'])
        test_model = copy.deepcopy(compressed_model)
        print("At relu slope:", relu, ":", test_accuracy)
    print("to be compressed_model", compressed_model)
    evaluate_model(compressed_model, dataloader, count_nonzero_only=True)
    w0 = compressed_model.blocks[layer_id].mlp_channels.fc1.weight.data
    w1 = compressed_model.blocks[layer_id].mlp_channels.fc2.weight.data
    b0 = compressed_model.blocks[layer_id].mlp_channels.fc1.bias.data
    b1 = compressed_model.blocks[layer_id].mlp_channels.fc2.bias.data
    compressed_model.blocks[layer_id].mlp_channels.fc2.weight.data = torch.matmul(w1, w0)
    compressed_model.blocks[layer_id].mlp_channels.fc2.bias.data = torch.matmul(w1, b0) + b1
    compressed_model.blocks[layer_id].mlp_channels.fc1 = nn.Identity()
    compressed_model.blocks[layer_id].mlp_channels.act = nn.Identity()
    compressed_model.blocks[layer_id].mlp_channels.drop1 = nn.Identity()
    evaluate_model(compressed_model, dataloader, count_nonzero_only=True)
    return compressed_model




In [6]:
from timm import models as models

In [7]:
prev_compressed_model = models.mlp_mixer.MlpMixer()


In [8]:
prev_compressed_model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [9]:
prev_compressed_model.load_state_dict(torch.load('MLP models/compressed_9_gelu.pt'))
# compressed_11_gelu_1.load_state_dict(torch.load('best_mlp_base.pt'))


<All keys matched successfully>

In [10]:
prev_compressed_model = prev_compressed_model.cuda()

In [11]:
evaluate_model(prev_compressed_model, dataloader, count_nonzero_only=True)

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=73.64%
model has size=196.90 MiB
model has macs=10.99 Gmacs
average inference time is 0.0031 seconds
model has 51.62 M parameters


In [12]:
mega_list = evaluate_topk(prev_compressed_model, dataloader['val'], topk=(5,))
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.9105813413858413


In [13]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [14]:
test_model = copy.deepcopy(prev_compressed_model)

In [15]:
compressed_model = collapse_mlp_gelu_layer_channel(test_model, 8, num_finetune_epochs=15)

test_model with leaky relu MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p

/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=73.00%
model has size=196.90 MiB
model has macs=10.99 Gmacs
average inference time is 0.0031 seconds
model has 51.62 M parameters
Finetuning changed Model


    Epoch 1 test Accuracy 73.46% / Best test Accuracy: 73.46%


    Epoch 2 test Accuracy 73.60% / Best test Accuracy: 73.60%


...

    Epoch 7 test Accuracy 71.72% / Best test Accuracy: 71.72%


    Epoch 8 test Accuracy 71.76% / Best test Accuracy: 71.76%


    Epoch 9 test Accuracy 71.18% / Best test Accuracy: 71.76%


    Epoch 10 test Accuracy 71.60% / Best test Accuracy: 71.76%


train:  87%|████████▋ | 434/501 [08:02<01:14,  1.11s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
evaluate_model(compressed_model, dataloader, count_nonzero_only=True)


/home/shariff/.local/lib/python3.8/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::gelu". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


model has test accuracy=71.76%
model has size=181.14 MiB
model has macs=10.18 Gmacs
average inference time is 0.0034 seconds
model has 47.48 M parameters


In [17]:
mega_list = evaluate_topk(prev_compressed_model, dataloader['val'], topk=(5,))
mega_list = [i[0].item() for i in mega_list]
print(sum(mega_list)/len(mega_list))

0.9105813413858413


In [18]:
torch.save(compressed_model.state_dict(), "MLP models/compressed_8_gelu.pt")
